<span class='note'><i>Make me look good.</i> Click on the cell below and press <kbd>Ctrl</kbd>+<kbd>Enter</kbd>.</span>

In [ ]:
from IPython.core.display import HTML
HTML(open('css/custom.css', 'r').read())

<h5 class='prehead'>SM286D &middot; Introduction to Applied Mathematics with Python &middot; Spring 2020 &middot; Uhan</h5>

<h5 class='lesson'>Lesson 18.</h5>

<h1 class='lesson_title'>More genetic algorithms, visualizing location data</h1>

## This lesson...

- Developing a genetic algorithm to solve the __traveling salesperson problem (TSP)__

- Visualizing location data with [**Cartopy**](https://scitools.org.uk/cartopy/docs/latest/)

---

In this lesson, we continue working with genetic algorithms. In the last lesson, we used a genetic algorithm to find a solution to the shortest path problem.  Today, we will develop a different genetic algorithm to find a solution to the __traveling salesperson problem (TSP)__.

The TSP is an important problem in operations research that is easy to state, but often difficult to solve in practice. The TSP asks the following question:

> Given a list of cities and the distances between each pair of cities, what is the shortest possible route that visits each city and returns to the origin city?

Finding efficient solutions to large scale instances of the TSP is important because the TSP has many different applications in planning, logistics, and manufacturing.  In slightly modified forms, the TSP can also appear as a subproblem in many different areas including biology (DNA sequencing) and astronomy (moving telescopes).

The problems below will guide you through the development of a genetic algorithm to solve the TSP.

---

## Classwork

**Problem 1.**
We will develop a genetic algorithm to approximately solve the TSP that visits each of the 61 US national parks. 

<img src="img/grandcanyon.jpg" width=350 />
<br>
<center>
    Figure 1. Wait, <em>what?</em> <a href="https://www.boredpanda.com/one-star-reviews-into-illustrations-national-parks-amber-share">[source]</a>
</center>

The names and locations of all the parks are stored in the Excel workbook `parks.xlsx`, in the same folder as this notebook. For each park, we have its name, its state (or states if the park spans several states), and its latitude and longitude. 

Use `xlwings` to load this data into the lists `parks`, `states`, `lats` and `longs`. 
Print a list of the parks to the screen. 

**Problem 2.**

We'd like to plot the locations of the parks on a map of the United States. We'll use the [**Cartopy**](https://scitools.org.uk/cartopy/docs/latest/) package to do this. 

First, we need to install Cartopy. Launch the Anaconda Prompt from the Windows Start Menu (bottom left corner of your screen). Then at the prompt, type 

```
conda install -c conda-forge cartopy
```

This only needs to be done the first time you want to use it. 

The code below produces a map of the United States with major waterways and borders. 

- Use the `ax.plot` method to add the parks to the plot. Recall (e.g., from Lesson 2) that `ax.plot` takes as input a list of $x$ coordinates and a list of $y$ coordinates. In this context, that translates to a list of longitudes and a list of latitudes (in that order).
- Use `ax.plot` options to mark the park locations with red dots.

In [ ]:
import cartopy 
import matplotlib.pyplot as plt

# Set up a Matplotlib figure and axes
# Put the map of the United States on these axes
# along with geographical features
fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=cartopy.crs.PlateCarree())
ax.add_feature(cartopy.feature.LAND)
ax.add_feature(cartopy.feature.OCEAN)
ax.add_feature(cartopy.feature.COASTLINE)
ax.add_feature(cartopy.feature.BORDERS, linestyle='-')
ax.add_feature(cartopy.feature.STATES, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, alpha=0.5)
ax.add_feature(cartopy.feature.RIVERS)
ax.set_extent([-175, -60, 0, 60])

##################################################
# Write your code here to plot the park locations
##################################################


# Show the plot
plt.show()

**Problem 3.**
We need to measure the distance between parks. Finding the distance between two points on Earth given by latitudes and longitudes is complicated. First, we need to convert each latitude and longitude from degrees to radians. Then, under the assumption that the Earth is a sphere of radius $R = 6371000$ meters, the distance between points $(long_1,lat_1)$ and $(long_2,lat_2)$ is given by

\begin{equation*}
R \arccos(\sin(lat_1)\sin(lat_2)+\cos(lat_1)\cos(lat_2)\cos(long_2-long_1)).
\end{equation*}

This distance is given in meters. We can convert it into miles by multiplying by the conversion factor `0.621371/1000`. 

Write a function `airdist` that takes as input two points given by latitude-longitude pairs (`lat1`, `long1`, `lat2`, `long2`), and returns the distance between the two points in miles.

_Hints._ 

- $d$ degrees is equal to $\frac{\pi d}{180}$ radians.
- Import `numpy` as `np` so you can use `np.sin` and other trigonometric functions, as well as `np.pi` for the value of $\pi$.
- Check your work. You should find the distance betwen Zion and Acadia National Parks to be about 2364 miles.

**Problem 4.**
Set `numparks` equal to the number of parks, 61. Don't do this manually, use `len`.

Use the `airdist` function to compute a $61 \times 61$ matrix named `distances` whose $(i,j)$ entry is the distance in miles between park number $i$ and park number $j$.
Write your code so that all diagonal entries of this matrix are exactly 0: sometimes Python's `arccos` function is inexact.

Check your work: print a sentence showing that the distance between parks 22 and 37 is about 2837.051.

**Problem 5.**
Since we want to visit each of the parks and return to our starting point, we can store the route that we follow as a __permutation__ of the numbers 0 to 60. We will represent permutations as lists.

For example, the permutation $[0, 24, 32, \dots, 1, 7, 5]$ would correspond to the route that starts at park 0, then goes to park 24, then park 32, and so on, until it reaches park 5. Then the route would go from park 5 back to park 0. This last step isn't included in our permutation, but it is always necessary to include the edge from the last park to the initial park in our route.

Write a function `circuitlength` that 
- takes as input a permutation of the numbers 0 through $n-1$ and an $n \times n$ distance matrix $d$, and 
- returns the length of the circuit corresponding to the permutation (that is, the route that we trace in moving along the route given by the permutation, including the final edge that returns us to the starting place).

Check your work: print a sentence confirming that the `circuitlength` of the permutation 

\begin{equation*}
[0,1,2,3,...,30,60,59,58,...,31]
\end{equation*}

is about 98268.31.

**Problem 6.**
We will use `circuitlength` from Problem 5 as the fitness function for our genetic algorithm. 

A parent consists of (1) a permutation $p$ of $[0,1,\ldots,60]$ and (2) a score, which is equal to the length $\ell$ of the corresponding circuit.
We start our genetic algorithm by generating 5 random parents. 

We want to choose the permutations randomly. However, it is also nice to be able to reproduce our results. So we set a __seed__ in the random number generator in Python, allowing us to run your code again and again, each time producing the same random output. (You'll learn more about random number generation and seeds in SA421 &mdash; Simulation Modeling.)

 - Read through the code below.
 - Print the last parent.

In [ ]:
# Number of parents
numparents = 5

# Set a seed for the random number generator
# Using the same seed results in the same random numbers generated
np.random.seed(210000)

# Create a list of parents
# Each parent is a dictionary consisting of 
# a permutation, and the length of the corresponding circuit
parents = []
for i in range(numparents):
    # Make a random permutation of the list containing 0, 1, ..., 60
    # Since np.random.permutation returns a NumPy array, 
    # we convert it to a list
    perm = list(np.random.permutation(list(range(numparks))))
    
    # Add the random permutation and corresponding circuit length
    parents.append({'perm': perm, 'score': circuitlength(perm, distances)})
    
# Write your code below
# Print the last parent
print(f"The last parent is {parents[-1]}.")

**Problem 7.**
The code below defines a function `invert_selection` that takes as input a permutation `perm`, an element `c` and a second element `cp`. The function returns a new permutation, which is obtained from `perm` by reversing the permutation from just after element `c` to just after element `cp`. For example, if 

<pre>
perm = [7, 2, 3, <span style="color:blue">9, 4, 1, 5,</span> 8, 6]
</pre>

and `c = 3` and `cp = 5`, then `invert_selection` returns

<pre>
newperm = [7, 2, 3, <span style="color:blue">5, 1, 4, 9,</span> 8, 6].
</pre>

In fact, a slightly different permutation is returned that gives an equivalent circuit to `newperm`: it follows the same tour but starts at a different element; this is irrelevant for our purposes. 

A second example shows that this isn't as simple as it might seem: if 

<pre>
perm = [<span style="color:blue">7, </span>2, 3, <span style="color:blue">9, 4, 1, 5, 8, 6</span>]
</pre>

and `c = 3` as before but `cp = 7` then we need to wrap around to get the output 

<pre>
newperm = [<span style="color:blue">9</span>, 2, 3, <span style="color:blue">7, 6, 8, 5, 1, 4</span>].
</pre>

Read through the definition of `invert_selection` below. Then test the code with the examples above and ensure that it works as advertised.

In [ ]:
def invert_selection(perm, c, cp):
    """
    Invert permutation perm between elements c and cp.
    """
    # We effectively rotate the permutation so that it starts 
    # just after c and ends at c.
    # This is helpful since it means that the portion of the 
    # permutation that we are reversing is at the start.

    # Length of permutation
    n = len(perm)
    
    # Double the permutation
    double_perm = perm + perm
    
    # Find the first index corresponding to c
    c_index = double_perm.index(c)
    
    # Rotate the permutation to start just after and end at c
    rotated_perm = double_perm[c_index + 1:c_index + n + 1]
    
    # Reverse the initial part of the rotated permutation,
    # between elements c and cp
    cp_index = rotated_perm.index(cp)
    
    newperm = rotated_perm[cp_index::-1] + rotated_perm[cp_index + 1::1]
    return newperm

In [ ]:
# Test the code in this code cell


**Problem 8.**
Implement a genetic algorithm for the TSP, using the following steps.

- Initialize the number of generations `num_gens` to 4000.
- In each generation, each parent produces one child. The child is a copy of the parent but two locations are exchanged:
    - Select a random index `a` from 0 to 59 (inclusive) &mdash; use `np.random.randint` to do this.
    - Next, select a random index `b` from `a + 1` to 60. 
    - Now create the child by swapping the values of the parent's permutation at index `a` and index `b`.
- Perform local improvement on each child:
    - Generate a random number between 0 and 1 &mdash; use `np.random.random` to do this.
    - If this number is less than 0.01, stop local improvement for this child.
    - If this number is at least 0.01, we randomly select another two indices `a` and `b` (with `a < b`), and invert the portion of the child's permutation starting just after the element with index `a` and continuing until just after the element with index `b`. 
        - Use the `invert_selection` function to do this. Note that the definition of that function calls for elements to be input into the function, not indices.
     - Repeat this process until local improvement is stopped.
- After the local improvement process stops, we compare the child to the parent. If the child's score is better than the parent's, we replace the parent with the child.

**Problem 9.**
Once your genetic algorithm has completed 4000 generations, print the fittest parent's route (i.e., permutation) as well as the length of the corresponding circuit.

**Problem 10.**
Print the *names* of the parks in the order in which you visit them (including one of the parks twice since you visit it first and then return to that park at the end of your journey).

**Problem 11.**
It would be nice to visualize what this tour looks like. Plot the resulting circuit (in black) on top of your figure from Problem 2.